In [ ]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
# -*- coding: utf-8 -*-
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#plt.ion()   # interactive mode 画图不阻止程序运行
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#显示中文字体设置
plt.rcParams["font.sans-serif"] = ["Droid Sans Fallback"]
plt.rcParams['axes.unicode_minus'] = False #为了正常显示是"-"减号


from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras
from keras.applications import *
#from keras.applications.vgg16 import preprocess_input, decode_prediction
from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import *
from keras.models import *

from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers

####################################################################
#设置可见的GPU数,注意不是并行训练的设置
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
#os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#看具体的模型参数设置在:https://www.tensorflow.org/api_docs/python/tf/keras/applications
MODELS = {'InceptionResNetV2':InceptionResNetV2}
#MODELS = {"DenseNet121":DenseNet121,"DenseNet201":DenseNet201,"InceptionResNetV2":InceptionResNetV2}
#"InceptionV3":InceptionV3,"DenseNet121":DenseNet121,
#       "DenseNet169":DenseNet169,"DenseNet201":DenseNet201,"Xception":Xception, 
#       "InceptionResNetV2":InceptionResNetV2,

classes = ['pant_length_labels','coat_length_labels','skirt_length_labels', 'sleeve_length_labels',] 
#classes = [ 'collar_design_labels','neckline_design_labels','skirt_length_labels', 
#            'sleeve_length_labels', 'neck_design_labels', 'lapel_design_labels', 
#            'pant_length_labels','coat_length_labels']   
#记录各分类的最高验证率 
fai_result = []
#不同参数设置不同版本
version = ""
#version = ["bestV1","bestV2","bestV3","bestV4","bestV5"]
label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}

####################################################################

attrs_cls_label_map = {
    'skirt_length_labels':['群不可见Invisible', '短群Short', '及膝群Knee', '旗袍裙群Midi', '及脚群Ankle', '接地群Floor'],
    'coat_length_labels': ['衣不可见Invisible','高腰衣HighWaistLength','常规衣RegularLength','长衣LongLength','加长衣MicroLength',
                           '及膝衣Knee Length','旗袍衣MidiLength','及地衣Ankle&FloorLength'],
    'collar_design_labels': ['衣领不可见Invisible','衬衫领ShirtCollar','彼得潘女士小圆领PeterPan','清道夫领PuritanCollar','螺纹领RibCollar'],
    'lapel_design_labels':['翻领不可见Invisible','缺口领Notched','无领Collarless','披肩围巾式领ShawlCollar','大号披肩围巾式领PlusSizeShawl'],
    'neck_design_labels':['脖颈不可见Invisible','长高领TurtleNeck', '荷叶半高领RuffleSemi-HighCollar','低圆领LowTurtleNeck','翻领Draped Collar'],
    'neckline_design_labels':['颈领线不可见Invisible','无肩带领StraplessNeck','深V领DeepVNeckline', '直领StraightNeck', 'V领VNeckline', 
                              '方领SquareNeckline', '出肩领OffShoulder', '圆领RoundNeckline', '桃形领SweatHeartNeck', '单肩领OneShoulderNeckline'],
    'pant_length_labels':[ '裤不可见Invisible', '短裤ShortPant', '中裤Mid Length', '7分裤3/4Length', '9分裤CroppedPant', '长裤FullLength'],
    'sleeve_length_labels':['袖不可见Invisible', '无袖Sleeveless', '杯袖CupSleeves', '短袖ShortSleeves', '肘中袖ElbowSleeves',
                            '7分袖Sleeves', '及腕9分袖WristLength', '长袖LongSleeves', '超长袖ExtraLongSleeves']  
}

######这是可视化预测输出函数
#predictor, 要测试的模型
#attr, 要测试的属性
#n_pictures, 要测试的图片数,最好别大于8,因为输出为1*n的形式
#image_width,该模型需要的图片尺寸

def fai_classify(predictor, attr, n_pictures, image_width):
    df = pd.read_csv('../train/Annotations/{0}.csv'.format(attr), header=None)
    df.columns = ['image_id', 'class', 'label']
    df_ = df.copy()
    df_.reset_index(inplace=True)
    del df_['index']
    n = len(df_)
    n_class = len(df_['label'][0])
    images_predict = np.zeros((n_pictures, image_width, image_width, 3), dtype=np.uint8)
    yy = np.zeros((n_pictures, n_class), dtype=np.uint8)
    print('产生随机索引序列')
    rdm = np.random.randint(n-1,size=n_pictures)
    true_label = np.zeros(n_pictures, dtype=np.uint8)
    for i in range(len(rdm)):
        j = rdm[i]
        images_predict[i] = cv2.resize(cv2.imread('../train/{0}'.format(df_['image_id'][j])), (image_width, image_width))
        tmp_label = df_['label'][j]
        true_label[i] = tmp_label.find('y')
        yy[i][tmp_label.find('y')] = 1
    print('装在数据和转化标签完毕,开始输入模型预测..')

    test_np = predictor.predict(images_predict, batch_size=n_pictures)
    aa = np.argmax(test_np, axis=1)
    fig,axes = plt.subplots(ncols=n_pictures,figsize=(4*n_pictures,4))  
    print('{0}张图片的真实与预测索引对比:'.format(n_pictures))
    print(true_label)
    print(aa)
    for i in range(n_pictures):
        axes[i].imshow(images_predict[i][:,:,::-1])
        j = true_label[i]
        k = aa[i]
        axes[i].set_title("pred-vis-{0},\nTrue:{2},\nPred:{3}".format(attr,
                        n_pictures, attrs_cls_label_map[attr][j], attrs_cls_label_map[attr][k]),color='r')  
    plt.show()
    #plt.title('{0}_predict_vis'.format(attr))
    print('完成测试!')
    #plt.savefig('../images/{0}/{0}_{1}_{2}.png'.format(prefix_cls, KEY, version),bbox_inches='tight')




In [ ]:
for KEY, MODLE in MODELS.items():
    
    print('######################在{0}下测试8个分类器####################'.format(KEY))
    for cur_class in classes:
        print('#######{0}:{1}####################'.format(KEY,cur_class ))
        #装载训练好保存的模型
        #
        pre_attr = cur_class.split('_')[0]
        model=load_model('../models/{0}/{0}_{1}v3.best.h5'.format(pre_attr, KEY),
            {'imagenet_utils':imagenet_utils})######当时在这不加这句dict就会出现imagenet_utils未定义,貌似是Lamda层的影响
        #解决办法:https://github.com/keras-team/keras/issues/4609,到处第3个,最下面有:
        
        print("模型装在完毕,开始测试!")
        #fai_classify(predictor, attr, n_pictures, image_width)
        fai_classify(model, cur_class, 5, 299)

print("completed!")